In [27]:
include("src/CombinedSubexpressions.jl")

CombinedSubexpressions

In [28]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

function h(x, y)
    println("h")
    3
end

x = 1

expr = quote
    function foo(x)
        [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
    end
end

quote  # In[28], line 19:
    function foo(x) # In[28], line 20:
        [h(h(f(g(x)),g(x)),i) == i for i = 1:5]
    end
end

In [29]:
CombinedSubexpressions.cacheify!(expr)

quote 
    begin  # In[28], line 19:
        function foo(x)
            ##gx#288 = g(x)
            ##f##gx#288#289 = f(##gx#288)
            ##h##f##gx#288#289##gx#288#290 = h(##f##gx#288#289,##gx#288)
            begin  # In[28], line 20:
                [h(##h##f##gx#288#289##gx#288#290,i) == i for i = 1:5]
            end
        end
    end
end

In [30]:
@CombinedSubexpressions.cse function foo(x)
    [f(x) == i for i in 1:5]
end

begin 
    function foo(x)
        ##fx#291 = f(x)
        begin  # In[30], line 2:
            [##fx#291 == i for i = 1:5]
        end
    end
end


foo (generic function with 1 method)

In [31]:
foo(1)

f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false

In [32]:
@CombinedSubexpressions.cse begin
    [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
end

begin 
    ##gx#292 = g(x)
    ##f##gx#292#293 = f(##gx#292)
    ##h##f##gx#292#293##gx#292#294 = h(##f##gx#292#293,##gx#292)
    begin  # In[32], line 2:
        [h(##h##f##gx#292#293##gx#292#294,i) == i for i = 1:5]
    end
end
g
f
h
h
h
h
h
h


5-element Array{Bool,1}:
 false
 false
  true
 false
 false

In [33]:
@CombinedSubexpressions.cse begin
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

begin 
    ##zeros5#295 = zeros(5)
    ##g5#296 = g(5)
    begin  # In[33], line 2:
        x = ##zeros5#295 # In[33], line 3:
        for i = 1:5 # In[33], line 4:
            x[i] = f(x[i]) + ##g5#296
        end
    end
end
g
f
f
f
f
f
